In [1]:
import geopandas as gpd
import pandas as pd
pd.set_option('display.max_columns', 80)
from shapely.geometry import Point, LineString
from geopandas import GeoDataFrame
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime

## Classifying investor purchases

Process to classify deeds as (a) residential properties, (b) likely purchased by investors.

## 1. Selecting deeds that fall within residential parcels

Using land use data from the city of Philadelphia (see original source [here](https://www.opendataphilly.org/dataset/land-use))

In [2]:
# read in land use file
landuse = gpd.read_file("data/Land_Use_2019/Land_Use.shp")

In [3]:
# creating a more narrow set of residential parcels
# just low and medium residential uses
res = landuse.loc[(landuse['C_DIG2DESC']=='Residential Medium') | 
                    (landuse['C_DIG2DESC']=='Residential Low')]

# dropping pacels with na values for their geometry
res = res.dropna(subset=['geometry'])

# reprojecting residential low/med only category
res = res.to_crs('epsg:2272')

In [4]:
# reading in the real estate tax transfer data
rtt = pd.read_csv("data/rtt_summary.csv")

/Users/adamvosburgh/opt/anaconda3/envs/geo-2021/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (26,34,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# selecting just deeds (not mortgages and other transfer taxes)
deeds = rtt.loc[rtt['document_type'] == "DEED"]
# adding a year column from the document date
deeds["sort_year"] = deeds['display_date'].astype(str).str[:4]

/Users/adamvosburgh/opt/anaconda3/envs/geo-2021/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [6]:
# defining points from lat/lon
deeds_points = [Point(xy) for xy in zip(deeds.lat,deeds.lng)]
deedsgeo = GeoDataFrame(deeds, geometry=deeds_points)

In [7]:
# selecting only deeds with non zero state taxes
deedsgeo = deedsgeo.loc[deedsgeo["state_tax_amount"] > 0]

# and that also have an address field
deedsgeo= deedsgeo.loc[deedsgeo["address_low"]>0]

In [8]:
# setting the projection for the lat lon coordinates 
deedsgeo.crs = 'epsg:4326'

In [9]:
# reprojecting to: EPSG:2272: NAD83 / Pennsylvania South (ftUS)
deedsgeo = deedsgeo.to_crs('epsg:2272')

In [10]:
# spatial join between deeds and residential parcels (to find only those deeds within residential parcels)
deeds_res = gpd.sjoin(deedsgeo,res, how="inner",op="within")

#deeds_res


## 2. Identify likely investor purchases
First tag all grantors with corporate entity names. Then find all investors who purchased more than 2 properties in any one year span.   

### 2A. Identifying Deeds with likely investors as the grantor based on grantor name
First tag all investor named entities   
  - LLC
  - LP
  - LIMITED PARTNERSHIP
  - CORP
  - CORPORATION
  - INC
  - COMPANY
  - LIMITED PARTNE
  - L P
  - L L C
  - PARTNERSHIP
  - LTD
  - TRUST 
    


In [11]:
# selecting just post-2000 deeds
deeds_res['sort_year'] = pd.to_numeric(deeds_res['sort_year'])
deeds_res = deeds_res.loc[deeds_res['sort_year']>1999]

# selecting investors by grantor name
inv = deeds_res['grantors'].str.contains(
    'LLC|LP|LIMITED PARTNERSHIP|CORP|CORPORATION|INC|COMPANY|LIMITED PARTNE|L P|L L C|PARTNERSHIP|LTD|TRUST'
    ,regex=True)

# converting investors series to dataframe
inv = inv.to_frame()

# merging with deeds
deeds_inv = deeds_res.merge(
    inv, left_index=True, right_index = True)
deeds_inv = deeds_inv.dropna(subset=['grantors_x'])

# creating a count column to allow grouping and count 
# for investor and owner by year 
deeds_inv['inv'] = deeds_inv['grantors_y'].apply(lambda x: 1 if x == True else 0)
deeds_inv['occ'] = deeds_inv['grantors_y'].apply(lambda x: 1 if x == False else 0)

# dropping duplicate object ids from the dataset
deeds_inv = deeds_inv.drop_duplicates(subset="objectid")

### 2B. Classifying investor sales by multiple purchasers
Identifying grantors who sold more than 2 properties in any one year period from the full set of non-investor classified deeds:

In [12]:
# finding multiple sellers from the full set of non-investor classified deeds
# selecting from just the non llc llp deeds
occ = deeds_inv.loc[deeds_inv['occ']==1]

In [13]:
# getting all deeds from sellers who are associated with more than one deed
multiple_purchasers = occ.groupby(
    "grantors_x").filter(lambda s: s['objectid'].count()>=2)


In [14]:
# converting date to datetime format
multiple_purchasers['display_date'] = multiple_purchasers[
    'display_date'].astype('datetime64[ns]')

In [15]:
# dropping unneeded columns
columns_multi = ['objectid','grantees','grantors_x',
                 'display_date','sort_year','street_address']
multiple_purchase_simple = multiple_purchasers[columns_multi]


In [25]:

multiple_purchase_simple[
    'elapsed'] = multiple_purchase_simple.sort_values(
    ['grantors_x','display_date']).groupby(
    'grantors_x')['display_date'].diff()

# converting timedelta column to an int type 
# to make it possible to filter below
multiple_purchase_simple[
    'elapsed_days'] = multiple_purchase_simple[
    'elapsed'].dt.days


/Users/adamvosburgh/opt/anaconda3/envs/geo-2021/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/adamvosburgh/opt/anaconda3/envs/geo-2021/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [26]:
# filtering for deeds from grantors who sold two properties within 1 year of each other
multiple_investors = multiple_purchase_simple.groupby('grantors_x').filter(
    lambda x: x['elapsed_days'].min() <= 365)
multiple_investors['multi_invest'] = 1

## 4. merging multiple purchase investors back with full set of deeds

Compiling a comprehensive classification of investor purchases 

In [27]:
deeds_multi = deeds_inv.merge(
    multiple_investors, left_on="objectid", right_on="objectid",how="left")

deeds_multi["grantor_investor_all"] = deeds_multi[
    "inv"] + deeds_multi["multi_invest"].fillna(0)
deeds_multi['owner_occup'] = deeds_multi[
    'grantor_investor_all'].apply(lambda x: 0 if x == 1 else 1)





In [28]:
# Saving classified deeds to speed up future processing time 
deeds_for_save = deeds_multi
deeds_for_save['elapsed'] = deeds_for_save['elapsed'].fillna(pd.Timedelta('999 days'))
deeds_for_save['elapsed'] = deeds_for_save['elapsed'].dt.days.astype('int16')
deeds_for_save['display_date_y'] = deeds_for_save['display_date_y'].astype(str)
deeds_for_save['grantor_inv'] = deeds_for_save['inv']
deeds_for_save['grantor_occ'] = deeds_for_save['occ']



In [29]:
# saving final list of grantors
with open("data/rtt_grantor-investors_classified.geojson",
          'w') as f:
    f.write(deeds_for_save.to_json())
    